In [27]:
import pandas as pd
from scipy.io import loadmat
from dtaidistance import dtw_ndim
from dtaidistance import dtw_ndim_visualisation
import dtw
from tqdm import tqdm
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn_extra.cluster import KMedoids

TRAFFIC数据集

In [34]:
data_m= loadmat("../TRAFFIC_dataset/TRAFFIC/TRAFFIC.mat")
print(data_m.keys())
tracks = data_m["tracks_traffic"]
#print(tracks.shape)
tracks=np.array(list(map(lambda x:x[0],tracks)))
#print(tracks.shape)
tracks=tracks.transpose([0,2,1])
labels = data_m["truth"][:, 0]#.reshape(1, -1)
labels = labels.flatten()
print(tracks.shape)
print(labels.shape)

dict_keys(['__header__', '__version__', '__globals__', 'truth', 'tracks_traffic'])
(300, 50, 2)
(300,)


In [15]:
dis_mat = np.zeros([tracks.shape[0], tracks.shape[0]])
with tqdm(total=tracks.shape[0]) as pbar1:
    for num1,track1 in enumerate(tracks):
        for num2,track2 in enumerate(tracks):
            if num1 < num2:
                dis=dtw_ndim.distance(track1, track2)
                dis_mat[num1,num2]=dis
                dis_mat[num2,num1]=dis
        pbar1.update(1)
np.save("TRAFFIC_dis_mat.npy",dis_mat)

100%|██████████| 300/300 [13:16<00:00,  2.66s/it]


In [5]:
#similarity = np.exp(-beta * distance / distance.std())
sim_mat = np.exp(-0.5 * dis_mat / dis_mat.std())
np.save("TRAFFIC_sim_mat.npy", sim_mat)

NameError: name 'dis_mat' is not defined

In [6]:
dis_mat=np.load("TRAFFIC_dis_mat.npy")
sim_mat=np.load("TRAFFIC_sim_mat.npy")

聚类算法实现

In [7]:
sse_max = -1
best_k = 0
sse_list = []
for k in range(2, 21):
    res = SpectralClustering(n_clusters=k, affinity='precomputed').fit(sim_mat)
    tmp = sklearn.metrics.silhouette_score(X=dis_mat, labels=res.labels_, metric='precomputed' )
    sse_list.append(tmp)
    if tmp > sse_max:
        sse_max = tmp
        best_k = k
best_res = SpectralClustering(n_clusters=best_k, affinity='precomputed').fit(sim_mat)
nmi = sklearn.metrics.normalized_mutual_info_score(labels_true=labels, labels_pred=best_res.labels_)
ari = sklearn.metrics.adjusted_rand_score(labels_true=labels, labels_pred=best_res.labels_)

print("sse_max:", sse_max)
print("best_k:", best_k)
print("nmi:", nmi)
print("ari:", ari)

sse_max: 0.6671851662748234
best_k: 11
nmi: 0.9783256534398461
ari: 0.9729666370986058


In [25]:
norm_dis_mat = dis_mat / (dis_mat.max() - dis_mat.min()) * 1000.0
#print(norm_dis_mat)
best_eps = -1
sse_max = -1
sse_list = []
for r in range(50, 200):
    res = DBSCAN(eps=r, metric='precomputed').fit(norm_dis_mat)
    #print(res.labels_)
    tmp = sklearn.metrics.silhouette_score(X=norm_dis_mat, labels=res.labels_, metric='precomputed' )
    sse_list.append(tmp)
    if tmp > sse_max:
        sse_max = tmp
        best_eps = r
nmi = sklearn.metrics.normalized_mutual_info_score(labels_true=labels, labels_pred=res.labels_)
ari = sklearn.metrics.adjusted_rand_score(labels_true=labels, labels_pred=res.labels_)
print("sse_max:", sse_max)
print("best_eps:", best_eps)
print("nmi:", nmi)
print("ari:", ari)

sse_max: 0.641360449247166
best_eps: 91
nmi: 0.8833128598889024
ari: 0.6956610137027762


In [28]:
sse_max = -1
best_k = 0
sse_list = []
for k in range(2, 21):
    res = KMedoids(n_clusters=k, metric='precomputed').fit(dis_mat)
    tmp = sklearn.metrics.silhouette_score(X=dis_mat, labels=res.labels_, metric='precomputed' )
    sse_list.append(tmp)
    if tmp > sse_max:
        sse_max = tmp
        best_k = k
best_res = KMedoids(n_clusters=best_k, metric='precomputed').fit(dis_mat)
nmi = sklearn.metrics.normalized_mutual_info_score(labels_true=labels, labels_pred=best_res.labels_)
ari = sklearn.metrics.adjusted_rand_score(labels_true=labels, labels_pred=best_res.labels_)

print("sse_max:", sse_max)
print("best_k:", best_k)
print("nmi:", nmi)
print("ari:", ari)

sse_max: 0.6059285686697852
best_k: 15
nmi: 0.9053089595463958
ari: 0.780678824240687
